In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
from tqdm import tqdm

import itertools
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

# 파생변수 생성 및 인코딩

In [3]:
## 나이 양수로 변환
train['birth_plus'] = train.DAYS_BIRTH*(-1)/365
test['birth_plus'] = test.DAYS_BIRTH*(-1)/365

train['gender']=LabelEncoder().fit_transform(train.gender)
train['car']=LabelEncoder().fit_transform(train.car)
train['reality']=LabelEncoder().fit_transform(train.reality)
train['income_type']=LabelEncoder().fit_transform(train.income_type)
train['edu_type']=LabelEncoder().fit_transform(train.edu_type)
train['family_type']=LabelEncoder().fit_transform(train.family_type)
train['house_type']=LabelEncoder().fit_transform(train.house_type)
train.fillna('NAN',inplace=True)
train['occyp_type']=LabelEncoder().fit_transform(train.occyp_type)
train['income_weight']=train.income_total**2
train['income_age']=train.income_total*train.DAYS_BIRTH
train['income_emp']=train.income_total*train.DAYS_EMPLOYED

test['gender']=LabelEncoder().fit_transform(test.gender)
test['car']=LabelEncoder().fit_transform(test.car)
test['reality']=LabelEncoder().fit_transform(test.reality)
test['income_type']=LabelEncoder().fit_transform(test.income_type)
test['edu_type']=LabelEncoder().fit_transform(test.edu_type)
test['family_type']=LabelEncoder().fit_transform(test.family_type)
test['house_type']=LabelEncoder().fit_transform(test.house_type)
test.fillna('NAN',inplace=True)
test['occyp_type']=LabelEncoder().fit_transform(test.occyp_type)
test['income_weight']=test.income_total**2
test['income_age']=test.income_total*test.DAYS_BIRTH
test['income_emp']=test.income_total*test.DAYS_EMPLOYED

train.income_total=train.income_total.astype(int)
train.family_size=train.family_size.astype(int)
train.begin_month=train.begin_month.astype(int)
train.birth_plus=train.birth_plus.astype(int)

test.income_total=test.income_total.astype(int)
test.family_size=test.family_size.astype(int)
test.begin_month=test.begin_month.astype(int)
test.birth_plus=test.birth_plus.astype(int)

## catboost는 categorical로 두면 안 됨
# train.credit=train.credit.astype('int')
## catboost는 categorical로 두면 안 됨
cat_credit=train.credit.astype('int')    

train.drop(['index','family_size','work_phone'],axis=1,inplace=True)
test.drop(['index','family_size','work_phone'],axis=1,inplace=True)
train.DAYS_BIRTH=train.DAYS_BIRTH**6
test.DAYS_BIRTH=test.DAYS_BIRTH**6
train.income_type=1/(train.income_type)
test.income_type=1/(test.income_type)
train.income_weight=1/(train.income_weight)**3
test.income_weight=1/(test.income_weight)**3
train['DAYS_BIRTH_weight']=1/(train.DAYS_BIRTH)**2
test['DAYS_BIRTH_weight']=1/(test.DAYS_BIRTH)**2

In [4]:
## predict_rf
ftr = train.drop('credit', axis=1).values
target = cat_credit.values
tst_ar = test.values
n_class = 3
n_fold = 10

In [6]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2020)
 
print(ftr.shape)
 
cb_p_val = np.zeros((ftr.shape[0], n_class))
cb_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    cb_clf = CatBoostClassifier(depth=8,
                                iterations=2000,
                                rsm=0.7,
                                ignored_features=5,
                                learning_rate=0.04,
                                l2_leaf_reg=1,
                               random_state = 2021,
                               verbose = 200)
    
    cb_clf.fit(ftr[i_trn], target[i_trn],
    eval_set=[(ftr[i_val], target[i_val])])
    
    cb_p_val[i_val, :] = cb_clf.predict_proba(ftr[i_val])
    cb_p_tst += cb_clf.predict_proba(tst_ar) / n_fold

    
print(f'{log_loss(target, cb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(cb_p_val, axis=1))}%')

(26457, 21)
training model for CV #1
0:	learn: 1.0755413	test: 1.0763213	best: 1.0763213 (0)	total: 15.1ms	remaining: 30.2s
200:	learn: 0.7155973	test: 0.7769310	best: 0.7769310 (200)	total: 3.16s	remaining: 28.3s
400:	learn: 0.6445615	test: 0.7518469	best: 0.7518469 (400)	total: 6.3s	remaining: 25.1s
600:	learn: 0.5891359	test: 0.7382821	best: 0.7382239 (589)	total: 9.48s	remaining: 22.1s
800:	learn: 0.5447184	test: 0.7288729	best: 0.7288403 (799)	total: 12.7s	remaining: 19s
1000:	learn: 0.5070674	test: 0.7234515	best: 0.7233211 (997)	total: 15.9s	remaining: 15.9s
1200:	learn: 0.4768229	test: 0.7207408	best: 0.7207408 (1200)	total: 19.1s	remaining: 12.7s
1400:	learn: 0.4488766	test: 0.7198555	best: 0.7193239 (1362)	total: 22.3s	remaining: 9.53s
1600:	learn: 0.4250172	test: 0.7201725	best: 0.7193239 (1362)	total: 25.6s	remaining: 6.37s
1800:	learn: 0.4036856	test: 0.7215727	best: 0.7193239 (1362)	total: 28.9s	remaining: 3.2s
1999:	learn: 0.3847605	test: 0.7237271	best: 0.7193239 (1362)

1000:	learn: 0.5124077	test: 0.7104202	best: 0.7103609 (998)	total: 15.7s	remaining: 15.6s
1200:	learn: 0.4811224	test: 0.7065734	best: 0.7064492 (1194)	total: 19s	remaining: 12.6s
1400:	learn: 0.4523707	test: 0.7054181	best: 0.7052899 (1385)	total: 22.1s	remaining: 9.44s
1600:	learn: 0.4281610	test: 0.7045396	best: 0.7045259 (1598)	total: 25.2s	remaining: 6.27s
1800:	learn: 0.4067327	test: 0.7060309	best: 0.7044704 (1604)	total: 28.2s	remaining: 3.12s
1999:	learn: 0.3874049	test: 0.7073164	best: 0.7044704 (1604)	total: 31.3s	remaining: 0us

bestTest = 0.7044704363
bestIteration = 1604

Shrink model to first 1605 iterations.
training model for CV #9
0:	learn: 1.0756126	test: 1.0760225	best: 1.0760225 (0)	total: 16.6ms	remaining: 33.2s
200:	learn: 0.7148648	test: 0.7726374	best: 0.7726374 (200)	total: 3.2s	remaining: 28.6s
400:	learn: 0.6437688	test: 0.7460334	best: 0.7460334 (400)	total: 6.28s	remaining: 25.1s
600:	learn: 0.5893684	test: 0.7303129	best: 0.7303129 (600)	total: 9.4s	rema

In [6]:
cb_p_val

array([[0.07092734, 0.25606068, 0.67301199],
       [0.7162199 , 0.05422348, 0.22955663],
       [0.07478861, 0.23266033, 0.69255106],
       ...,
       [0.17117013, 0.127546  , 0.70128387],
       [0.14513153, 0.05602456, 0.79884391],
       [0.03722257, 0.48693288, 0.47584455]])

In [35]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'cat'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, cb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, cb_p_tst, fmt='%.6f', delimiter=',')

In [62]:
print(f'{log_loss(target, cb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(cb_p_val, axis=1))}%')

0.6811907370591721
[[  897   435  1890]
 [  221  2867  3179]
 [  414   782 15772]]%


----

In [66]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'lgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
sample_submission=pd.read_csv('data/sample_submission.csv')
for i in range(17):
    sample_submission.iloc[:,1:]+=models_lgbm[i].predict_proba(test)/17
    
sample_submission.to_csv('state2021.csv',index=False)

In [29]:
feature_lgbm=feature_lgbm.sort_index()
feature_lgbm

print(np.mean(np.array(logloss_avg)))

0.6880214218100895


In [30]:
print(np.mean(np.array(logloss_avg)))

0.6880214218100895


- 90 : 0.6890796591713713

----

In [28]:
############################################################################
label=train.credit
train=train.drop('credit',axis=1)
skf=StratifiedKFold(n_splits=17,random_state=40,shuffle=True)
folds=[]
for train_idx,val_idx in skf.split(train0,label):
    folds.append([train_idx,val_idx])
models_lgbm={}
logloss_avg=[]
feature_lgbm=pd.DataFrame()
for i in range(17):
    train_x,val_x,train_y,val_y=train0.iloc[folds[i][0],:],train0.iloc[folds[i][1],:],label[folds[i][0]],label[folds[i][1]]
    lgb=LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=99,
                       verbose = 50
                       # reg_alpha=0.1
                    )
    lgb.fit(train_x,train_y)
    logloss_avg.append(log_loss(val_y,lgb.predict_proba(val_x)))
    models_lgbm[i]=lgb
    feature_lgbm=pd.concat([feature_lgbm,pd.DataFrame(lgb.predict_proba(val_x),index=folds[i][1])],axis=0)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001282 seconds, init for row-wise cost 0.001238 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Re-bagging, using 22402 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22422 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Deb

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22462 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22376 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22430 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22393 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22406 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22472 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Traine

...

[LightGBM] [Debug] Re-bagging, using 22451 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22408 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22459 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22385 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22443 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Deb